In [ ]:
!pip install vedo trimesh open3d


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.0 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attem

In [ ]:
import numpy as np
import vedo
import trimesh
import open3d as o3d
import random

# Paso 1: Crear puntos aleatorios
num_puntos = 10
puntos = np.random.uniform(-5, 5, size=(num_puntos, 3))

# Listas para cada sistema
objetos_vedo = []
trimesh_objs = []
open3d_meshes = []

# Colores aleatorios (R,G,B)
def color_rgb():
    return [random.random() for _ in range(3)]

for i, punto in enumerate(puntos):
    x, y, z = punto
    size = np.random.uniform(0.5, 1.5)
    color = color_rgb()

    forma = i % 3  # 0: esfera, 1: cubo, 2: cilindro

    # === VEDO ===
    if forma == 0:
        obj_v = vedo.Sphere(pos=punto, r=size, c=color)
        trimesh_obj = trimesh.creation.icosphere(radius=size)
    elif forma == 1:
        obj_v = vedo.Cube(pos=punto, side=size, c=color)
        trimesh_obj = trimesh.creation.box(extents=[size]*3)
    else:
        obj_v = vedo.Cylinder(pos=punto, r=size/2, height=size*2, axis=(0,0,1), c=color)
        trimesh_obj = trimesh.creation.cylinder(radius=size/2, height=size*2)

    objetos_vedo.append(obj_v)

    # === TRIMESH ===
    trimesh_obj.visual.vertex_colors = (np.array(color) * 255).astype(np.uint8)
    trimesh_obj.apply_translation(punto)
    trimesh_objs.append(trimesh_obj)

    # === OPEN3D ===
    mesh_o3d = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(trimesh_obj.vertices),
        triangles=o3d.utility.Vector3iVector(trimesh_obj.faces)
    )
    mesh_o3d.compute_vertex_normals()
    mesh_o3d.paint_uniform_color(color)  # Pintar color uniforme
    open3d_meshes.append(mesh_o3d)

# === EXPORTACIÓN ===

# vedo
from vedo import merge
merged = merge(objetos_vedo)
vedo.write(merged, "scene_vedo.obj")  # .obj/.stl también válidos

# trimesh
scene_trimesh = trimesh.Scene(trimesh_objs)
scene_trimesh.export("scene_trimesh.glb")  # glTF soporta color

# open3d
scene_o3d = open3d_meshes[0]
for m in open3d_meshes[1:]:
    scene_o3d += m
o3d.io.write_triangle_mesh("scene_open3d.obj", scene_o3d)  # PLY guarda color

# Vista previa (opcional en Colab)
vedo.show(objetos_vedo, axes=1, title="Escena Aleatoria 3D")


In [ ]:
mesh_open3d = trimesh.load("scene_open3d.obj", force='mesh')
mesh_trimesh = trimesh.load("scene_trimesh.glb", force='mesh')
mesh_vedo = trimesh.load("scene_vedo.obj", force='mesh')

In [ ]:
mesh_open3d.show()

In [ ]:
mesh_trimesh.show()

In [ ]:
mesh_vedo.show()